# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [91]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [92]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [93]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [94]:
totals = data.groupby(['CustomerID','ProductName'],as_index=False).agg({'Quantity':'sum'})
totals

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [95]:
pivot = pd.pivot_table(data, values='Quantity', index='ProductName', columns='CustomerID', aggfunc='sum',fill_value=0)
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [96]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))),index=pivot.columns, columns=pivot.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [97]:
similarities = distances[33].sort_values(ascending=False)[1:]
similar = pd.DataFrame(similarities[:5])
similar

,33
CustomerID,
264,0.087047
3535,0.087047
3317,0.087047
2503,0.085983
3305,0.085638


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [98]:
custID = list(similar.index)
tot = totals[totals['CustomerID'].isin(custID)].reset_index(drop=True)
tot

,CustomerID,ProductName,Quantity
0,264,Apricots - Halves,1
1,264,Apricots Fresh,1
2,264,Bacardi Breezer - Tropical,1
3,264,Bagel - Plain,1
4,264,Banana - Leaves,1
...,...,...,...
265,3535,Tray - 16in Rnd Blk,1
266,3535,Wanton Wrap,1
267,3535,Wine - Blue Nun Qualitatswein,1
268,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [99]:
group = tot.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index(drop=True)
group

,ProductName,Quantity
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
2,Towels - Paper / Kraft,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3
...,...,...
213,Hinge W Undercut,1
214,Ice Cream Bar - Hageen Daz To,1
215,Jagermeister,1
216,Jolt Cola - Electric Blue,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [111]:
merge = group.merge(pivot, on='ProductName')

filt = merge[merge[33]==0]
filt.drop('index',axis=1,inplace=True)
lst = list(filt.iloc[:,3:].columns)
filt.drop(lst, axis=1, inplace=True)
filt.head()

C:\Users\irock\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,ProductName,Quantity,33
0,Butter - Unsalted,3,0
1,Wine - Ej Gallo Sierra Valley,3,0
3,Soup - Campbells Bean Medley,3,0
4,Wine - Blue Nun Qualitatswein,3,0
6,Chicken - Soup Base,2,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [115]:
recom = dict()

In [117]:
lst.insert(0, 33)

In [118]:
lst

[33,
 200,
 264,
 356,
 412,
 464,
 477,
 639,
 649,
 669,
 694,
 756,
 883,
 891,
 1008,
 1034,
 1066,
 1072,
 1336,
 1428,
 1435,
 1534,
 1577,
 1594,
 1754,
 1839,
 1920,
 2187,
 2329,
 2503,
 2556,
 2566,
 2582,
 2617,
 2686,
 2754,
 2776,
 2902,
 2915,
 2939,
 3074,
 3253,
 3267,
 3305,
 3317,
 3472,
 3531,
 3535,
 3544,
 3885,
 3903,
 3909,
 4261,
 4479,
 4595,
 4644,
 4723,
 5104,
 5177,
 5183,
 5224,
 5230,
 5434,
 5678,
 5739,
 5832,
 5921,
 5968,
 5986,
 5997,
 6001,
 6185,
 6317,
 6352,
 6367,
 6447,
 6543,
 6734,
 6817,
 6851,
 7021,
 7134,
 7223,
 7543,
 7606,
 7623,
 7772,
 7827,
 7987,
 7999,
 8316,
 8367,
 8372,
 8539,
 8711,
 8917,
 8962,
 9047,
 9132,
 9260,
 9302,
 9533,
 9546,
 9597,
 9771,
 9804,
 10016,
 10094,
 10114,
 10224,
 10311,
 10418,
 10739,
 11023,
 11057,
 11083,
 11102,
 11253,
 11325,
 11380,
 11441,
 11443,
 11614,
 11628,
 11744,
 11883,
 11933,
 12059,
 12206,
 12232,
 12381,
 12529,
 12561,
 12596,
 12598,
 12852,
 12888,
 13062,
 13118,
 13338,
 

In [129]:
for i in lst:
    similarities = distances[i].sort_values(ascending=False)[1:]
    similar = pd.DataFrame(similarities[:5])
    custID = list(similar.index)

    tot = totals[totals['CustomerID'].isin(custID)].reset_index(drop=True)

    group = tot.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index(drop=True)

    merge = group.merge(pivot[i], on='ProductName')

    filt = merge[merge[i]==0]

    recom[i] = list(filt['ProductName'].head())


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [140]:
df = pd.DataFrame(recom)
df=df.T.reset_index()
df.rename(columns={'index':'CustomerID'},inplace=True)
df

,CustomerID,0,1,2,3,4
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
2,264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
3,356,"Veal - Inside, Choice",Lamb - Ground,Pomello,Butter - Unsalted,"Tart Shells - Sweet, 4"
4,412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley
...,...,...,...,...,...,...
995,97928,"Soup - Campbells, Lentil","Cheese - Brie,danish",Wiberg Super Cure,Pants Custom Dry Clean,Shrimp - 31/40
996,98069,Skirt - 29 Foot,Beans - Kidney White,Milk - 1%,Cheese - Taleggio D.o.p.,Sprouts - Baby Pea Tendrils
997,98159,"Lamb - Whole, Fresh",Bread - Raisin Walnut Oval,Chips Potato Salt Vinegar 43g,Tea - English Breakfast,Wine - Redchard Merritt
998,98185,Crackers - Trio,Pernod,Tea - Jasmin Green,Pastry - Choclate Baked,"Peas - Pigeon, Dry"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [141]:
distances2 = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'correlation'))),index=pivot.columns, columns=pivot.columns)
for i in lst:
    similarities = distances2[i].sort_values(ascending=False)[1:]
    similar = pd.DataFrame(similarities[:5])
    custID = list(similar.index)

    tot = totals[totals['CustomerID'].isin(custID)].reset_index(drop=True)

    group = tot.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index(drop=True)

    merge = group.merge(pivot[i], on='ProductName')

    filt = merge[merge[i]==0]

    recom[i] = list(filt['ProductName'].head())

In [144]:
df = pd.DataFrame(recom)
df=df.T.reset_index()
df.rename(columns={'index':'CustomerID'},inplace=True)
df

,CustomerID,0,1,2,3,4
0,33,Knife Plastic - White,Muffin - Zero Transfat,Banana Turning,Crush - Cream Soda,Veal - Osso Bucco
1,200,Otomegusa Dashi Konbu,Milk Powder,Potatoes - Idaho 100 Count,Crackers - Trio,Pail With Metal Handle 16l White
2,264,"Water - Mineral, Natural",Wine - Toasted Head,Snapple - Iced Tea Peach,Pickerel - Fillets,Garbag Bags - Black
3,356,Cheese - Taleggio D.o.p.,"Coconut - Shredded, Sweet",Cheese - Cheddarsliced,Ocean Spray - Kiwi Strawberry,Olives - Kalamata
4,412,Cake - Mini Cheesecake,Butter - Unsalted,"Salmon - Atlantic, Skin On",Wine - Hardys Bankside Shiraz,Gloves - Goldtouch Disposable
...,...,...,...,...,...,...
995,97928,Lettuce - Treviso,"Yogurt - Blueberry, 175 Gr",Bread - Calabrese Baguette,Extract - Lemon,Coffee - Irish Cream
996,98069,"Wine - Red, Colio Cabernet",Veal - Inside,Soupfoamcont12oz 112con,Peas - Frozen,Sprouts - Baby Pea Tendrils
997,98159,"Water, Tap",Bananas,Wine - Redchard Merritt,Chocolate - Compound Coating,Longos - Grilled Salmon With Bbq
998,98185,Cheese - Taleggio D.o.p.,Squid U5 - Thailand,"Peas - Pigeon, Dry",Salmon Steak - Cohoe 8 Oz,Veal - Osso Bucco
